In [11]:
import re
import pandas as pd # CSV file I/O (pd.read_csv)
from nltk.corpus import stopwords

import copy
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [12]:
news = pd.read_excel("C:\\Users\\Kratika\\Downloads\\News data (Completed) (1) (9).xlsx")
news.head()

,Title,Body,Category
0,Adani Ports Shares Jump Nearly 6% as Buyback O...,"August 26, 2019, 4:25 PM IST\nFor Representati...",Buyback of shares - Alert me when any of these...
1,Adani Ports buyback to open on September 6; sh...,Adani Ports and Special Economic Zone\ngained ...,Buyback of shares - Alert me when any of these...
2,Adani Ports Q1 net profit jumps 47.50%,Adani Ports and Special Economic Zone (APSEZ) ...,"Earnings, Stock Performance or Dividends - Ale..."
3,Adani Ports keen on building port at Nagarjuna...,Adani Ports and Special Economic Zone Ltd (APS...,New Geography - Alert me when a company moves ...
4,Adani Ports Q1 result: Adani Ports Q1 PAT jump...,Adani Ports\non Wednesday said its consolidate...,"Earnings, Stock Performance or Dividends - Ale..."


In [13]:
from nltk.tokenize import word_tokenize 

def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(data) 
    filtered_sentence = []
    
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    
    return " ".join(filtered_sentence)

In [14]:
remove_stop_words_Body = news['Body'].apply(remove_stop_words) 
#remove_stop_words_Body

In [32]:
# ## Split data
print("\nSplitting data")
title_tr, title_te, category_tr, category_te = train_test_split(remove_stop_words_Body, news.Category,test_size =.02)
title_tr, title_de, category_tr, category_de = train_test_split(title_tr,category_tr,test_size =.02)
print("Training: ",len(title_tr))
print("Developement: ",len(title_de),)
print("Testing: ",len(title_te))


Splitting data
Training:  2398
Developement:  49
Testing:  50


In [33]:
from sklearn.preprocessing import LabelEncoder

# # Data Preprocessing
# ## Vectorization of data
# Vectorize the data using Bag of words (BOW)
print("\nVectorizing data")
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
stop_words = nltk.corpus.stopwords.words("english")
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)

vectorizer.fit(iter(title_tr))
Xtr = vectorizer.transform(iter(title_tr))
Xde = vectorizer.transform(iter(title_de))
Xte = vectorizer.transform(iter(title_te))

encoder = LabelEncoder()
encoder.fit(category_tr)
Ytr = encoder.transform(category_tr)
Yde = encoder.transform(category_de)
Yte = encoder.transform(category_te)


Vectorizing data


In [34]:
# ## Feature Reduction
# We can check the variance of the feature and drop them based on a threshold
print("\nApplyting Feature Reduction")
print("Number of features before reduction : ", Xtr.shape[1])
selection = VarianceThreshold(threshold=0.001)
Xtr_whole = copy.deepcopy(Xtr)
Ytr_whole = copy.deepcopy(Ytr)
selection.fit(Xtr)
Xtr = selection.transform(Xtr)
Xde = selection.transform(Xde)
Xte = selection.transform(Xte)
print("Number of features after reduction : ", Xtr.shape[1])


Applyting Feature Reduction
Number of features before reduction :  16601
Number of features after reduction :  7379


In [35]:


# # Train Models
# ### Baseline Model
# “stratified”: generates predictions by respecting the training set’s class distribution.
print("\n\nTraining baseline classifier")
dc = DummyClassifier(strategy="stratified")
dc.fit(Xtr, Ytr)
pred = dc.predict(Xde)






Training baseline classifier


In [36]:
print(classification_report(Yde, pred))

              precision    recall  f1-score   support

           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
          10       0.12      0.12      0.12         8
          16       0.00      0.00      0.00         0
          19       0.00      0.00      0.00         0
          20       0.00      0.00      0.00         0
          21       0.00      0.00      0.00         0
          24       0.00      0.00      0.00         1
          25       0.00      0.00      0.00         1
          27       0.00      0.00      0.00         0
          31       0.00      0.00      0.00         1
          32       0.00      0.00      0.00         1
          34       0.00      0.00      0.00         2
          36       0.00      0.00      0.00         0
          37       0.33      0.20      0.25         5
          40       0.00      0.00      0.00         3
          42       0.00    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
